Downloaded 3 files frmo citibike site (August, September, October). Want to create a union here to basically stack them on top of each other vertically, as opposed to a join that would horizontally attach them.

In [1]:
# Import dependencies
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine

In [2]:
# Read the CSV files

# October file
oct_csv = pd.read_csv("Resources/JC-202410-citibike-tripdata.csv")

# September file
sept_csv = pd.read_csv("Resources/JC-202409-citibike-tripdata.csv")

# August file
aug_csv = pd.read_csv("Resources/JC-202408-citibike-tripdata.csv")

In [3]:
# Concatenate the dfs vertically
combined_df = pd.concat([aug_csv, sept_csv, oct_csv], ignore_index=True)

# Display new df
combined_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,17AE31FCAE74D287,electric_bike,2024-08-07 13:22:55.656,2024-08-07 13:25:09.654,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member
1,FD9859BDBE0CDF70,electric_bike,2024-08-13 13:15:08.627,2024-08-13 13:17:44.971,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member
2,AAC5ECD095AE5572,electric_bike,2024-08-12 20:07:26.975,2024-08-12 20:09:38.180,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member
3,857C4DCB2F29655B,electric_bike,2024-08-09 13:43:18.882,2024-08-09 13:45:38.226,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member
4,4439657C244E7009,classic_bike,2024-08-01 10:29:40.174,2024-08-01 10:32:56.874,Clinton St & Newark St,HB409,4 St & Grand St,HB301,40.737430,-74.035710,40.742258,-74.035111,member


In [4]:
len(combined_df['end_lng'])

340316

Now that we've combined them, we need to save them to CSV from Tableau.

In [41]:
# Save to CSV
combined_df.to_csv("Resources/combined_citibike_data.csv", index=False)

We need to combine this CSV with a combined GeoJSON containing both the New Jersey and New York data.

In [5]:
# Load JSON files to a geopandas dataframe? 
gdf_nj = gpd.read_file('Resources/nj_cut.json')
gdf_ny = gpd.read_file('Resources/nyc_cut.json')

In [6]:
# Examine first 5 rows of NJ gdf
gdf_nj.head()

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,34,07430,3407430,B5,G6350,S,66496320,1290236,+41.0831588,-074.1858947,N,"POLYGON ((-74.10379 41.08690, -74.11229 41.090..."
1,34,07650,3407650,B5,G6350,S,3203430,63688,+40.8470173,-073.9970615,N,"POLYGON ((-73.99133 40.83588, -73.98220 40.845..."
2,34,07652,3407652,B5,G6350,S,27117162,128793,+40.9473094,-074.0709890,N,"POLYGON ((-74.08932 40.91964, -74.08512 40.918..."
3,34,07072,3407072,B5,G6350,S,10357741,631509,+40.8264242,-074.0623383,N,"POLYGON ((-74.10277 40.84148, -74.09813 40.838..."
4,34,07661,3407661,B5,G6350,S,4785271,104542,+40.9273540,-074.0396108,N,"POLYGON ((-74.03884 40.94369, -74.04025 40.940..."


In [7]:
# Rename columns
gdf_nj.rename(columns={'STATEFP10': 'State','ZCTA5CE10': 'Zipcode'}, inplace=True)

gdf_nj.head()

,State,Zipcode,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,34,07430,3407430,B5,G6350,S,66496320,1290236,+41.0831588,-074.1858947,N,"POLYGON ((-74.10379 41.08690, -74.11229 41.090..."
1,34,07650,3407650,B5,G6350,S,3203430,63688,+40.8470173,-073.9970615,N,"POLYGON ((-73.99133 40.83588, -73.98220 40.845..."
2,34,07652,3407652,B5,G6350,S,27117162,128793,+40.9473094,-074.0709890,N,"POLYGON ((-74.08932 40.91964, -74.08512 40.918..."
3,34,07072,3407072,B5,G6350,S,10357741,631509,+40.8264242,-074.0623383,N,"POLYGON ((-74.10277 40.84148, -74.09813 40.838..."
4,34,07661,3407661,B5,G6350,S,4785271,104542,+40.9273540,-074.0396108,N,"POLYGON ((-74.03884 40.94369, -74.04025 40.940..."


In [8]:
gdf_ny.head()

,STATEFP10,ZCTA5CE10,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,36,10464,3610464,B5,G6350,S,9070627,236605,+40.8677868,-073.7999204,N,"MULTIPOLYGON (((-73.78487 40.86027, -73.78755 ..."
1,36,10470,3610470,B5,G6350,S,3689950,3918,+40.8895273,-073.8726596,N,"POLYGON ((-73.85268 40.90990, -73.85488 40.908..."
2,36,10455,3610455,B5,G6350,S,1844518,0,+40.8147100,-073.9085917,N,"POLYGON ((-73.91195 40.82015, -73.91742 40.821..."
3,36,10473,3610473,B5,G6350,S,5645604,13868,+40.8186904,-073.8584741,N,"POLYGON ((-73.88433 40.82261, -73.88055 40.818..."
4,36,11234,3611234,B5,G6350,S,19547849,2143727,+40.6050798,-073.9117206,N,"POLYGON ((-73.93184 40.58292, -73.92830 40.583..."


In [9]:
# Rename columns
gdf_ny.rename(columns={'STATEFP10': 'State','ZCTA5CE10': 'Zipcode'}, inplace=True)

gdf_ny.head()

,State,Zipcode,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,36,10464,3610464,B5,G6350,S,9070627,236605,+40.8677868,-073.7999204,N,"MULTIPOLYGON (((-73.78487 40.86027, -73.78755 ..."
1,36,10470,3610470,B5,G6350,S,3689950,3918,+40.8895273,-073.8726596,N,"POLYGON ((-73.85268 40.90990, -73.85488 40.908..."
2,36,10455,3610455,B5,G6350,S,1844518,0,+40.8147100,-073.9085917,N,"POLYGON ((-73.91195 40.82015, -73.91742 40.821..."
3,36,10473,3610473,B5,G6350,S,5645604,13868,+40.8186904,-073.8584741,N,"POLYGON ((-73.88433 40.82261, -73.88055 40.818..."
4,36,11234,3611234,B5,G6350,S,19547849,2143727,+40.6050798,-073.9117206,N,"POLYGON ((-73.93184 40.58292, -73.92830 40.583..."


In [10]:
gdf_combined = pd.concat([gdf_nj, gdf_ny], ignore_index=True)

Now the GeoJSON files have been concatenated in a data frame `gdf_combined`.

In [11]:
gdf_combined.head()

,State,Zipcode,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,34,07430,3407430,B5,G6350,S,66496320,1290236,+41.0831588,-074.1858947,N,"POLYGON ((-74.10379 41.08690, -74.11229 41.090..."
1,34,07650,3407650,B5,G6350,S,3203430,63688,+40.8470173,-073.9970615,N,"POLYGON ((-73.99133 40.83588, -73.98220 40.845..."
2,34,07652,3407652,B5,G6350,S,27117162,128793,+40.9473094,-074.0709890,N,"POLYGON ((-74.08932 40.91964, -74.08512 40.918..."
3,34,07072,3407072,B5,G6350,S,10357741,631509,+40.8264242,-074.0623383,N,"POLYGON ((-74.10277 40.84148, -74.09813 40.838..."
4,34,07661,3407661,B5,G6350,S,4785271,104542,+40.9273540,-074.0396108,N,"POLYGON ((-74.03884 40.94369, -74.04025 40.940..."


Let's try exporting this `gdf_combined` as a geojson, and see if the join in Tableau will work better

In [12]:
# Verify if `gdf_merged` is a GeoDataFrame
print(type(gdf_combined))

<class 'geopandas.geodataframe.GeoDataFrame'>


In [52]:
# Export to GeoJSON
gdf_combined.to_file('Output/citibike_NYNJ.geojson', driver='GeoJSON')

Now we have `gdf_combined` to merge with `combined_citibike_data.csv`. Hopefully this will give zipcode data to every row of the CSV.

In [13]:
combined_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,17AE31FCAE74D287,electric_bike,2024-08-07 13:22:55.656,2024-08-07 13:25:09.654,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member
1,FD9859BDBE0CDF70,electric_bike,2024-08-13 13:15:08.627,2024-08-13 13:17:44.971,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member
2,AAC5ECD095AE5572,electric_bike,2024-08-12 20:07:26.975,2024-08-12 20:09:38.180,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member
3,857C4DCB2F29655B,electric_bike,2024-08-09 13:43:18.882,2024-08-09 13:45:38.226,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member
4,4439657C244E7009,classic_bike,2024-08-01 10:29:40.174,2024-08-01 10:32:56.874,Clinton St & Newark St,HB409,4 St & Grand St,HB301,40.737430,-74.035710,40.742258,-74.035111,member


In [15]:
# Convert CSV lat/lng into a GeoDataFrame
gdf_points = gpd.GeoDataFrame(combined_df, 
                               geometry=gpd.points_from_xy(combined_df['end_lng'], combined_df['end_lat']),
                               crs="EPSG:4326")


In [16]:
gdf_points.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,geometry
0,17AE31FCAE74D287,electric_bike,2024-08-07 13:22:55.656,2024-08-07 13:25:09.654,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member,POINT (-74.03511 40.74226)
1,FD9859BDBE0CDF70,electric_bike,2024-08-13 13:15:08.627,2024-08-13 13:17:44.971,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member,POINT (-74.03511 40.74226)
2,AAC5ECD095AE5572,electric_bike,2024-08-12 20:07:26.975,2024-08-12 20:09:38.180,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member,POINT (-74.03511 40.74226)
3,857C4DCB2F29655B,electric_bike,2024-08-09 13:43:18.882,2024-08-09 13:45:38.226,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member,POINT (-74.03511 40.74226)
4,4439657C244E7009,classic_bike,2024-08-01 10:29:40.174,2024-08-01 10:32:56.874,Clinton St & Newark St,HB409,4 St & Grand St,HB301,40.737430,-74.035710,40.742258,-74.035111,member,POINT (-74.03511 40.74226)


In [17]:
gdf_points.dtypes

ride_id                 object
rideable_type           object
started_at              object
ended_at                object
start_station_name      object
start_station_id        object
end_station_name        object
end_station_id          object
start_lat              float64
start_lng              float64
end_lat                float64
end_lng                float64
member_casual           object
geometry              geometry
dtype: object

In [18]:
gdf_combined.head()

,State,Zipcode,GEOID10,CLASSFP10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,PARTFLG10,geometry
0,34,07430,3407430,B5,G6350,S,66496320,1290236,+41.0831588,-074.1858947,N,"POLYGON ((-74.10379 41.08690, -74.11229 41.090..."
1,34,07650,3407650,B5,G6350,S,3203430,63688,+40.8470173,-073.9970615,N,"POLYGON ((-73.99133 40.83588, -73.98220 40.845..."
2,34,07652,3407652,B5,G6350,S,27117162,128793,+40.9473094,-074.0709890,N,"POLYGON ((-74.08932 40.91964, -74.08512 40.918..."
3,34,07072,3407072,B5,G6350,S,10357741,631509,+40.8264242,-074.0623383,N,"POLYGON ((-74.10277 40.84148, -74.09813 40.838..."
4,34,07661,3407661,B5,G6350,S,4785271,104542,+40.9273540,-074.0396108,N,"POLYGON ((-74.03884 40.94369, -74.04025 40.940..."


In [19]:
# Perform a spatial join to add the Zipcode information from gdf_combined
gdf_merged_concat = gpd.sjoin(gdf_points, gdf_combined[['geometry', 'Zipcode']], how="left", op='within')

c:\Users\nbart.DESKTOP-3OF7M8N\anaconda3\envs\dev\lib\site-packages\IPython\core\interactiveshell.py:3517: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [20]:
# It now has a Zipcode column... that could be huge for Tableau
gdf_merged_concat.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,geometry,index_right,Zipcode
0,17AE31FCAE74D287,electric_bike,2024-08-07 13:22:55.656,2024-08-07 13:25:09.654,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member,POINT (-74.03511 40.74226),162.0,07030
1,FD9859BDBE0CDF70,electric_bike,2024-08-13 13:15:08.627,2024-08-13 13:17:44.971,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member,POINT (-74.03511 40.74226),162.0,07030
2,AAC5ECD095AE5572,electric_bike,2024-08-12 20:07:26.975,2024-08-12 20:09:38.180,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member,POINT (-74.03511 40.74226),162.0,07030
3,857C4DCB2F29655B,electric_bike,2024-08-09 13:43:18.882,2024-08-09 13:45:38.226,7 St & Monroe St,HB304,4 St & Grand St,HB301,40.746413,-74.037977,40.742258,-74.035111,member,POINT (-74.03511 40.74226),162.0,07030
4,4439657C244E7009,classic_bike,2024-08-01 10:29:40.174,2024-08-01 10:32:56.874,Clinton St & Newark St,HB409,4 St & Grand St,HB301,40.737430,-74.035710,40.742258,-74.035111,member,POINT (-74.03511 40.74226),162.0,07030


In [21]:
# Verify if `gdf_merged` is a GeoDataFrame
print(type(gdf_merged_concat))

<class 'geopandas.geodataframe.GeoDataFrame'>


Verifying that the Geomtery will work in Tableau. Many hours wrestling with issues related to maps and geography. 

In [61]:
# Export to CSV
gdf_merged_concat.to_csv('Output/citibike_merged.csv')

This didn't work correctly and the only to make it work was a Tableau join with `citibike_NYNJ.geojson`. After that full outer join the Zipcode polygon layer finally worked.